In [1]:
import numpy as np
import joblib

%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

import sys
sys.path.append("../")
sys.path.append("../../RARinterpret")
import lgpc
import RARinterpret

%load_ext autoreload
%autoreload 2


_X, y, weights, features, indxs = (joblib.load('../../RARinterpret/results/processed_cat.p')[p]
                           for p in ("X", "y", "weights", "features", "indxs"))
x = _X[:, 2]
z = _X[:, 0]

from scipy.stats import spearmanr

from scipy.stats import truncnorm, norm

from sklearn.linear_model import LinearRegression

In [2]:
n_splits = 10
# GroupShuffle split mask
test_masks = RARinterpret.train_test_shuffle_mask(indxs, n_splits=n_splits, test_size=42,
                                                  random_state=42)


# gpr = GaussianProcessRegressor(kernel=RBF() + WhiteKernel(), n_restarts_optimizer=1, normalize_y=True)

gpr = LinearRegression()



dxz, dyz, fullout = lgpc.get_reg_residuals(gpr, x, y, z, test_masks, partial=False)

100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 891.19it/s]


In [3]:
dxz_summ = lgpc.fold_average(dxz)

dyz_summ = lgpc.fold_average(dyz)

In [4]:
p = np.linspace(z.min(), z.max(), 100)
cf, ps = lgpc.local_partial_correlation(dxz_summ, dyz_summ, p=z, peval=p, width=0.2, Nrepeat=30,
                                        kernel="gaussian")

NameError: name 'stats' is not defined

In [ ]:
k = 0
mu = cf[:, k, 0]
std = cf[:, k, 1]

fig, ax = plt.subplots(nrows=2, sharex=True, gridspec_kw={"height_ratios": [1, 0.5]})
fig.subplots_adjust(hspace=0)
ax[0].plot(p, mu)
ax[0].fill_between(p, mu - std, mu + std, alpha=0.5)

ax[0].plot(p, ps[:, k])
ax[0].axhline(0, lw=1)

# ax[0].plot(zeval, np.median(out[..., 1], axis=1))

ax[1].hist(z, bins="auto")


plt.show()

In [ ]:
mu = np.mean(out[..., 0], axis=1)
std = np.std(out[..., 0], axis=1)

fig, ax = plt.subplots(nrows=2, sharex=True, gridspec_kw={"height_ratios": [1, 0.25]})
fig.subplots_adjust(hspace=0)
ax[0].plot(zeval, mu)
ax[0].fill_between(zeval, mu - std, mu + std, alpha=0.5)
# ax[0].plot(zeval, np.median(out[..., 1], axis=1))

ax[1].hist(z, bins="auto")


plt.show()

In [ ]:


plt.figure()

m = (z > 1.5) & (z < 2)

plt.scatter(x, y, s=3, c=z)
plt.colorbar()

# plt.scatter(z, x, s=1)
# plt.scatter(z, y, s=1)

plt.show()

In [ ]:
features

In [ ]:
lgpc